# Brownian Motion

In [ ]:
using Pkg; Pkg.activate()
using KadanoffBaym
using LinearAlgebra

In [ ]:
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")
using LaTeXStrings

In [ ]:
function solution(N_0::Float64, D::Float64; args...)
    function F_ana(t1, t2)
        return (F[1, 1] - D/(2 * theta)) * exp(-theta * (t1 + t2)) + D/(2 * theta) * exp(-theta * abs(t1 - t2))
    end
    
    function fv!(out, times, t1, t2)
        out[1] = -theta * F[t1, t2]
    end
    
    function fd!(out, times, t1, t2)
        out[1] = -theta * 2F[t1, t2] + D
    end
    
    F = GreenFunction(N_0 * ones(1,1), Symmetrical)
    
    sol = kbsolve!(fv!, fd!, [F], (0.0, T); args...)

    return [sol.t, F.data, [F_ana(t1, t2) for t1 in sol.t, t2 in sol.t]]
end;

In [ ]:
T = 4.0
theta = 1.
t_scale = (iszero(theta) ? one(theta) : abs(theta))

N_0 = [1., 3., 5.]
Ds = [8., 4., 1.]

s = mapreduce((x...) -> solution(x...; atol=1e-9, rtol=1e-7, dtini=1e-10), hcat, N_0, Ds);

## Plotting

In [ ]:
cmap = "gist_heat";
colors = ["C0", "C1", "C2"];
lss = ["-", "--", "-."];

In [ ]:
fig = figure(figsize=(7, 3))

ax = subplot(121)
for k in eachindex(N_0)
    plot(s[1,k], diag(s[2,k]), label="\$xxx\$", lw=1.5, c=colors[k], ls=lss[k])
end
ax.set_xlim(0, theta * T)
ax.set_ylim(0, 1.1 * Ds[1]/(2theta))
ax.set_yticks([0, 1, 2, 3, 4, 5])
ax.set_xlabel(L"\theta t")
ax.set_title(L"F(t, t)")

ax = subplot(122)
plot([], [], label=L"D/\theta", c="w")
for k in eachindex(N_0)
    semilogy(s[1,k], abs.((diag(s[2,k])  .- diag(s[3,k])) ./ diag(s[3,k])), lw=1.5, c=colors[k], label=L"%$(string(Ds[k]/(theta)))", ls=lss[k])
end

ax.set_xlim(0, theta * T)
ax.set_ylim(1e-9, 1e-6)
ax.set_xlabel(L"\theta t")
ax.set_title(L"\left|\frac{F(t,t) - \mathcal{F}(t,t)}{\mathcal{F}(t,t)}\right|")
ax.yaxis.set_ticks_position("right")
ax.yaxis.set_label_position("right")

ax.legend(loc="best", frameon=false, labelspacing=0.0, borderpad=0, handlelength=1.5, fontsize="small")
tight_layout(pad=0.25, w_pad=1, h_pad=0)

# savefig("brownian_motion_example_1.pdf")

In [ ]:
function meshgrid(xin,yin)
  nx=length(xin)
  ny=length(yin)
  xout=zeros(ny,nx)
  yout=zeros(ny,nx)
  for jx=1:nx
      for ix=1:ny
          xout[ix,jx]=xin[jx]
          yout[ix,jx]=yin[ix]
      end
  end
  return (x=xout, y=yout)
end

In [ ]:
Y, X = meshgrid(s[1,1], s[1,1]);

In [ ]:
figure(figsize=(7, 3))

vmin = 0
vmax = Ds[1]/(2theta)
ax = subplot(121) # plt.gca()
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, s[2,1], cmap=cmap, rasterized=true, vmin=vmin, vmax=vmax)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel(L"\theta t")
ax.set_ylabel(L"\theta t^\prime")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])
ax.set_title(L"F(t, t)")

ax = subplot(122)
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, abs.((s[2,1] - s[3,1]) ./ s[3,1]), cmap="gist_gray", rasterized=true, vmin=0e-6, vmax=3e-6)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel(L"\theta t")
# ax.set_ylabel("\$\\theta t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])
ax.set_yticklabels([])
ax.set_title(L"\left|\frac{F(t,t) - \mathcal{F}(t,t)}{\mathcal{F}(t,t)}\right|")

tight_layout(pad=0.75, w_pad=0.25, h_pad=0)
# savefig("brownian_motion_example_2.pdf")

## Adaptive vs Fixed dt

In [ ]:
T = 0.5

In [ ]:
s = mapreduce((x...) -> solution(x...; atol=0, rtol=1e-4, dtini=2^(-7), kmax=4), hcat, N_0, Ds);
idx = 1
dts = s[1, idx][2:end] .- s[1, idx][1:end-1];

In [ ]:
@show error = let
  e = abs.((s[2, idx] - s[3, idx]) ./ s[3, idx])
  e = diag(e)
  sum(e) / *(size(e)...)
end

In [ ]:
figure(figsize=(5, 3))
semilogy(s[1, idx][2:end], dts, "o-")
xlim(0, theta * T)
ylim(1e-3, 1e-1)
xlabel(L"\theta t")
ylabel(L"\mathrm{d} t")
tight_layout(pad=0.25, w_pad=1, h_pad=0)
# savefig("brownian_motion_adaptive_dt.pdf")

In [ ]:
s = mapreduce((x...) -> solution(x..., rtol=1e0, dtmax=2^(-7), dtini=2^(-7), kmax=4, γ=1), hcat, N_0, Ds);
idx = 1
dts = s[1, idx][2:end] .- s[1, idx][1:end-1];

In [ ]:
figure(figsize=(5, 3))
semilogy(s[1, idx][2:end], dts, "o-")
xlim(0, theta * T)
ylim(1e-3, 1e-1)
xlabel(L"\theta t")
ylabel(L"\mathrm{d} t")
tight_layout(pad=0.25, w_pad=1, h_pad=0)
# savefig("brownian_motion_fixed_dt.pdf")

In [ ]:
@show error = let
  e = abs.((s[2, idx] - s[3, idx]) ./ s[3, idx])
  e = diag(e)
  sum(e) / *(size(e)...)
end